<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/color_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')
df.head(3)

,Unnamed: 0,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn,drive_img_path,pixel
0,0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...,/content/drive/MyDrive/kream_data/image/28029.jpg,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1,1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...,/content/drive/MyDrive/kream_data/image/12831.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...
2,2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 ...,/content/drive/MyDrive/kream_data/image/21935.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...


In [5]:
df.shape

(6103, 15)

In [6]:
color_input_data = np.array([])

for val in df['tokenized_color'].values:
    val = [int(x) for x in val.strip('[]').split()]
    color_input_data = np.append(color_input_data, val)

color_input_data = color_input_data.reshape([6103, 14])
color_input_data = color_input_data.astype(int)

In [7]:
color_input_data.shape

(6103, 14)

In [8]:
color_input_data

array([[ 0,  0,  0, ...,  4,  3,  5],
       [ 0,  0,  0, ...,  4,  3,  4],
       [ 0,  0,  0, ..., 82,  3, 51],
       ...,
       [ 0,  0,  0, ...,  5,  3,  4],
       [ 0,  0,  0, ...,  4,  3,  4],
       [ 0,  0,  0, ...,  4,  3,  4]])

In [9]:
columns = [x for x in range(1000)]
index = [y for y in range(color_input_data.shape[0])]


color_input_df = pd.DataFrame(index=index, columns=columns)
color_input_df = color_input_df.fillna(0)

for index, line in enumerate(color_input_data):
    for elem in set(line):
        if elem < 1000:
            color_input_df.iloc[index,elem] = 1
        else:
            color_input_df.iloc[index,0] = 1

In [10]:
color_input_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 정가를 피처에 추가해준다.

color_input_df = pd.concat([color_input_df, df["price_og"]], axis=1)

In [12]:
color_input_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,price_og
0,1,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,129000
1,1,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,139000
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,169000


In [13]:
color_input_df.shape

(6103, 1001)

In [14]:
# 신발의 브랜드와 이름은 리셀가격(신발의 본래 가치??)와 관련이 클 것 같아서 target데이터를 price_resell 로 설정함. 

target_data = np.array(df['price_resell'])
print(target_data.shape)

(6103,)


In [15]:
target_data.astype(int)

array([152800, 136200, 173200, ..., 450000, 457500, 400000])

In [16]:
for n in [10,30,50]:
    pca = PCA(n_components=n)
    color_pca = pca.fit_transform(color_input_df)
    color_pca_df = pd.DataFrame(color_pca)
    print(sum(pca.explained_variance_ratio_))

0.9999999999350974
0.9999999999581246
0.9999999999669896


In [17]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 

for n in [30, 50]:
    pca = PCA(n_components=n)
    color_pca = pca.fit_transform(color_input_df)
    color_pca_df = pd.DataFrame(color_pca)

    kf = KFold(n_splits = 5, shuffle=True)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        color_pca_df_train, color_pca_df_test = color_pca_df.loc[train_index], color_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        reg = LinearRegression().fit(color_pca_df_train, target_data_train)

        pred = reg.predict(color_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 30 rmse: 286546
PCA: 50 rmse: 286504


In [18]:
# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 

from lightgbm import LGBMRegressor

# PCA의 차원을 늘리면서  K-fold를 사용해서 어떤 모델이 가장 효율적인지 살펴보겠다. 

for n in [10, 30, 50]:
    pca = PCA(n_components=n)
    color_pca = pca.fit_transform(color_input_df)
    color_pca_df = pd.DataFrame(color_pca)

    kf = KFold(n_splits = 5, shuffle=True)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        color_pca_df_train, color_pca_df_test = color_pca_df.loc[train_index], color_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        lgbm_reg = LGBMRegressor().fit(color_pca_df_train, target_data_train)

        pred = lgbm_reg.predict(color_pca_df_test)
        rmse = mean_squared_error(pred, target_data_test, squared=False)
        rmse_list.append(rmse)
        # print(rmse)
        
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 10 rmse: 276059
PCA: 30 rmse: 274748
PCA: 50 rmse: 272262


In [19]:
for n in [32, 64]:
    pca = PCA(n_components=n)
    color_pca = pca.fit_transform(color_input_df)
    color_pca_df = pd.DataFrame(color_pca)

    kf = KFold(n_splits = 5, shuffle=True)
    
    rmse_list = []
    for train_index, test_index in kf.split(target_data):
        color_pca_df_train, color_pca_df_test = color_pca_df.loc[train_index], color_pca_df.loc[test_index]
        target_data_train, target_data_test  = target_data[train_index], target_data[test_index]

        model = keras.models.Sequential([
                keras.layers.Dense(16),
                keras.layers.Dense(8),
                keras.layers.Dense(1)
        ])

        model.compile(loss="mse")
        model.fit(color_pca_df_train,target_data_train, epochs=20, verbose=0)
        pred = model.predict(color_pca_df_test, verbose=0)
        rmse = mean_squared_error(target_data_test,pred, squared= False)

        rmse_list.append(rmse)
    print("PCA:", n, "rmse:", round(np.mean(rmse_list)))

PCA: 32 rmse: 378961
PCA: 64 rmse: 379309


여러 가지를 실험해봤는데.... bn_dnn과 마찬가지로

- 주성분을 30개~50개로 축소한 lgbm 모델이 적절할듯 하다.

뭔가 rmse가 너무 큰데.... 잘 모르겠다.

일단 이미지인식 CNN까지 진행하고 생각해 봐야겠다.


In [31]:
color_input_df.isna().sum().value_counts()

0    1001
dtype: int64

In [36]:
for i in np.isnan(target_data):
    if i:
        print("nan")

In [53]:
df['price_resell'] /= 10000.0
target_data = df['price_resell']

In [63]:
from keras.models import Sequential
from tensorflow import keras

model1 = Sequential([
    keras.layers.Dense(64),
    keras.layers.Dense(32),
    keras.layers.Dense(16),
    keras.layers.Dense(8),
    keras.layers.Dense(4),
    keras.layers.Dense(2),
    keras.layers.Dense(1)
 ])

model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss=tf.keras.losses.Huber(delta=3))
checkpoint_cb = keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/kream_data/model/color_dnn.h5")

model1.fit(color_input_df, target_data, epochs=100, callbacks=[checkpoint_cb])

Epoch 1/100
191/191 [==============================] - 57s 6ms/step - loss: 1530.4161
Epoch 2/100
191/191 [==============================] - 1s 4ms/step - loss: 57.1749
Epoch 3/100
191/191 [==============================] - 1s 4ms/step - loss: 36.4331
Epoch 4/100
191/191 [==============================] - 1s 3ms/step - loss: 33.4547
Epoch 5/100
191/191 [==============================] - 1s 3ms/step - loss: 34.2319
Epoch 6/100
191/191 [==============================] - 1s 4ms/step - loss: 31.3565
Epoch 7/100
191/191 [==============================] - 1s 4ms/step - loss: 30.4070
Epoch 8/100
191/191 [==============================] - 1s 4ms/step - loss: 30.3570
Epoch 9/100
191/191 [==============================] - 1s 5ms/step - loss: 35.8584
Epoch 10/100
191/191 [==============================] - 1s 5ms/step - loss: 283.0996
Epoch 11/100
191/191 [==============================] - 1s 5ms/step - loss: 29.7338
Epoch 12/100
191/191 [==============================] - 1s 4ms/step - loss: 29.70

In [64]:
pred1 = model1.predict(color_input_df)
mse1 = mean_squared_error(pred1, target_data, squared=False) 

mse1

191/191 [==============================] - 0s 1ms/step


30.04250335943222

In [65]:
model = keras.models.load_model("/content/drive/MyDrive/kream_data/model/color_dnn.h5") 
model.evaluate(color_input_df, df['price_resell'])

191/191 [==============================] - 1s 2ms/step - loss: 29.8949


29.894895553588867

In [68]:
model.predict(color_input_df.iloc[0])

InvalidArgumentError: ignored

In [71]:
color_input_df.iloc[0, :]

0                1
1                0
2                1
3                1
4                1
             ...  
996              0
997              0
998              0
999              0
price_og    129000
Name: 0, Length: 1001, dtype: int64